In [1]:
import os

# 设置环境变量
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 打印环境变量以确认设置成功
print(os.environ.get('HF_ENDPOINT'))

https://hf-mirror.com


In [2]:
from transformers import AutoTokenizer, AutoConfig,AutoModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import  AutoConfig, AutoModelForCausalLM,LlamaForCausalLM,LlamaTokenizer
from tokenizers import Tokenizer
from datasets import load_dataset

2024-11-24 11:21:51.020375: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 11:21:51.036615: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 11:21:51.053557: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 11:21:51.058466: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 11:21:51.071840: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from datasets import load_dataset
dna_ft_dataset = load_dataset('json', data_files='val_data.json')
dna_ft_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 19839
    })
})

In [4]:
data = dna_ft_dataset["train"].train_test_split(train_size=0.1, seed=42)
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1983
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 17856
    })
})

In [5]:
tokenizer = LlamaTokenizer.from_pretrained("dnahlm-llama-7b-sft-v0") #dnagpt/dnahlm-llama-7b-sft-v0
tokenizer.pad_token = tokenizer.eos_token

In [6]:
model = LlamaForCausalLM.from_pretrained("dnahlm-llama-7b-sft-v0") #continue pretrain
model

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(61973, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      

In [7]:
#构建提示词
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text + "\n\n### Response:\n"

#构建提示词
def build_prompt(entry):

    input_data = format_input(entry)

    desired_response = entry['output']

    return input_data + desired_response



In [8]:
example = data["test"][0]
example

{'instruction': 'Determine core promoter detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.',
 'input': 'CCGTGCGACCGGAAGTGGGGCGGCGACCCCGGAAGTCCCCGCCGGGTGCAGCTTGGTCGGTTCGATCGCC',
 'output': 'promoter'}

In [9]:
prompt = build_prompt(example)
print(prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine core promoter detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.

### Input:
CCGTGCGACCGGAAGTGGGGCGGCGACCCCGGAAGTCCCCGCCGGGTGCAGCTTGGTCGGTTCGATCGCC

### Response:
promoter


In [10]:
tokenizer.tokenize(prompt)

['▁Below',
 '▁is',
 '▁an',
 '▁instruction',
 '▁that',
 '▁describes',
 '▁a',
 '▁task',
 '.',
 '▁Write',
 '▁a',
 '▁response',
 '▁that',
 '▁appropri',
 'ately',
 '▁comple',
 'tes',
 '▁the',
 '▁request',
 '.',
 '<0x0A>',
 '<0x0A>',
 '##',
 '#',
 '▁Inst',
 'ruction',
 ':',
 '<0x0A>',
 'Det',
 'erm',
 'ine',
 '▁core',
 '▁prom',
 'oter',
 '▁detection',
 '▁of',
 '▁following',
 '▁d',
 'na',
 '▁sequence',
 ',',
 '▁The',
 '▁result',
 '▁will',
 '▁be',
 '▁one',
 '▁of',
 '▁the',
 '▁following',
 ':',
 '▁Non',
 '-',
 'prom',
 'oter',
 ',',
 '▁prom',
 'oter',
 '.',
 '<0x0A>',
 '<0x0A>',
 '##',
 '#',
 '▁Input',
 ':',
 '<0x0A>',
 'CCGTG',
 'C',
 'GAC',
 'CGGAA',
 'GTG',
 'GGGC',
 'GGC',
 'GAC',
 'CCCGGAA',
 'GTCC',
 'CCGCC',
 'GGGTG',
 'CA',
 'GCT',
 'TG',
 'GTC',
 'GGT',
 'TC',
 'GATCGCC',
 '<0x0A>',
 '<0x0A>',
 '##',
 '#',
 '▁Response',
 ':',
 '<0x0A>',
 'prom',
 'oter']

In [11]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=1000):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
          # return_attention_mask=True,
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    #max_length=max_output_tokens,
    max_new_tokens=8,
    temperature=0.01  # 控制生成的多样性
  )

  # Decode
  generated_text_with_prompt = tokenizer.decode(generated_tokens_with_prompt[0], skip_special_tokens=True)
  generated_text_answer = generated_text_with_prompt[len(text):]


  return generated_text_answer

# 如果需要进一步清理
def clean_generated_text(text):
    # 去除 'Ġ' 符号并替换为空格
    text = text.replace('Ġ', ' ')
    # 去除多余的空格
    text = ' '.join(text.split())
    return text

In [12]:
input_text = format_input(data["test"][0])

print("input (test):", input_text)

print("real answer:", data["test"][0]["output"])

print("--------------------------\n")

print("model's answer: \n")
print(inference(input_text, model, tokenizer))

input (test): Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Determine core promoter detection of following dna sequence, The result will be one of the following: Non-promoter, promoter.

### Input:
CCGTGCGACCGGAAGTGGGGCGGCGACCCCGGAAGTCCCCGCCGGGTGCAGCTTGGTCGGTTCGATCGCC

### Response:

real answer: promoter
--------------------------

model's answer: 



/root/miniconda3/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


 Non-promoter


In [13]:
test_data = data["test"].shuffle(seed=199).select(range(1000))

data_list = []

for entry in test_data:
    input_text = format_input(entry)
    #print(input_text)
    response_text = inference(input_text, model, tokenizer)
    #print(response_text)
    data = {
        "instruction":entry["instruction"],
         "input":entry["input"],
         "output":entry["output"],
        "model_response":response_text
    }

    data_list.append(data)

In [14]:
import json

# 定义输出文件路径
output_file = 'llama-sft-2.json'

# 将 Dataset 对象导出为 JSON 文件
# test_data.to_json(output_file)
with open(output_file, "w") as file:
    json.dump(data_list, file, indent=4)  # "indent" for pretty-printing



In [15]:
" ".join(tokenizer.tokenize("promoter"))

'▁prom oter'

In [16]:
import json
from tqdm import tqdm



with open(output_file, "r") as file:
    test_data = json.load(file)

all_num = len(test_data)
right_sum = 0
same_sum = 0
for item in test_data:
    output = item["output"]
    #output = " ".join(tokenizer.tokenize(output))
    model_response = item["model_response"]

    print(output,"||||||||||||", model_response)

    if model_response == output: #same it
        same_sum = same_sum + 1
        
    if output.find("Non")==-1: # no Non
        if model_response.find(output)!=-1 and model_response.find("Non")==-1: #find it, but no Non
            right_sum = right_sum + 1
    else:
        if model_response.find(output)!=-1: #find it
            right_sum = right_sum + 1


print("presicion", right_sum/all_num, "same", same_sum/all_num)


Donor Sites ||||||||||||  Donor Sites
promoter |||||||||||| promoter
promoter |||||||||||| promoter
promoter |||||||||||| promoter
promoter |||||||||||| promoter
Donor Sites ||||||||||||  Non-Splice Sites
promoter ||||||||||||  Non-promoter
promoter ||||||||||||  Non-promoter
Non-promoter |||||||||||| promoter
Non-promoter ||||||||||||  Non-promoter
Donor Sites ||||||||||||  Donor Sites
Non-promoter ||||||||||||  Non-promoter
Non-promoter |||||||||||| promoter
Non-promoter |||||||||||| promoter
promoter ||||||||||||  Non-promoter
promoter ||||||||||||  Non-promoter
Donor Sites ||||||||||||  Donor Sites
Background Sequences ||||||||||||  Background Sequences
Non-promoter ||||||||||||  Non-promoter
Non-promoter |||||||||||| promoter
promoter |||||||||||| promoter
promoter |||||||||||| promoter
promoter |||||||||||| promoter
promoter |||||||||||| promoter
promoter ||||||||||||  Non-promoter
promoter ||||||||||||  promoter
Non-promoter ||||||||||||  Non-promoter
Non-Splice Sites ||||||||||